In [ ]:
import h5py as h5
import numpy as np

from glob import glob
from tqdm import tqdm

from scipy.interpolate import CubicSpline

import matplotlib.pyplot as plt
plt.style.use("../paper.mplstyle")
from matplotlib.colors import ListedColormap, to_rgba, to_rgb
from matplotlib.gridspec import GridSpec
from matplotlib.lines import Line2D

import sys
import os

sys.path.append("../../ASTERIA/python/")
os.environ["ASTERIA"] = "/Users/jlazar/research/ASTERIA"

sys.path.append("../")
from sne_bsm import parameterized_flux_from_files, units
from sne_bsm.likelihood import find_significance_matrix 
from sne_bsm.flux.parametrize_flux import deserialize

In [ ]:
SM_SERIALIZE_FILE = "./data/serialized_sm_flux.h5"
BSM_SERIALIZE_FILE = "./data/serialized_majoran_fluxes.h5"

MODEL1 = "dphi-dEdt-1MeV-gn10dot2-100s-nf"
MODEL2 = "dphi-dEdt-10MeV-gn10dot8-100s-nf"
MODEL3 = "dphi-dEdt-100MeV-gn11dot7-100s-nf"

MODELS = [
    MODEL1,
    # MODEL2,
    MODEL3,
]

COLORS = [
    "mediumturquoise", # MODEL1
    "dodgerblue", # MODEL2
    "hotpink",
    # "red", # SM
]

LABELS = [
    r"$m_{\phi}=1~\mathrm{MeV}$",
    r"$m_{\phi}=100~\mathrm{MeV}$",
]

# Flux plot

## Deserialize the fluxes

In [ ]:
with h5.File(SM_SERIALIZE_FILE, "r") as h5f:
    sm_flux = deserialize(h5f["sm_flux_0"])

bsm_fluxes = []

for model in tqdm(MODELS):
# for model in tqdm([MODEL1, MODEL2, MODEL3]):
    with h5.File(BSM_SERIALIZE_FILE, "r") as h5f:
        flux = deserialize(h5f[f"{model}_0"])
    bsm_fluxes.append(flux)

## Make the plot

In [ ]:
es = np.linspace(0, 200, 400) * units.MeV
ts = np.array([0.1, 0.3, 1, 3, 10]) * units["second"]

alphamin = 0.3
alphamax = 1.0

handles = [
    Line2D([], [], color=COLORS[-1], label=r"SM"),
    Line2D([], [], color=COLORS[0], label=r"$m=1~\mathrm{MeV}$"),
    # Line2D([], [], color=COLORS[1], label=r"$m=10~\mathrm{MeV}$"),
    Line2D([], [], color=COLORS[1], label=r"$m=100~\mathrm{MeV}$"),
    Line2D([], [], alpha=0.0),
    Line2D([], [], alpha=0.0),
    Line2D([], [], color="k", alpha=0/4*(alphamax - alphamin) + alphamin, label=r"$\mathrm{0.1}~\mathrm{s}$"),
    Line2D([], [], color="k", alpha=1/4*(alphamax - alphamin) + alphamin, label=r"$\mathrm{0.3}~\mathrm{s}$"),
    Line2D([], [], color="k", alpha=2/4*(alphamax - alphamin) + alphamin, label=r"$\mathrm{1}~\mathrm{s}$"),
    Line2D([], [], color="k", alpha=3/4*(alphamax - alphamin) + alphamin, label=r"$\mathrm{3}~\mathrm{s}$"),
    Line2D([], [], color="k", alpha=4/4*(alphamax - alphamin) + alphamin, label=r"$\mathrm{10}~\mathrm{s}$"),
]


for idx, t in enumerate(ts):
    alpha = idx / (len(ts)-1) * (alphamax - alphamin) + alphamin
    for color, flux in zip(COLORS, bsm_fluxes):
        plt.plot(
            es / units.MeV,
            flux.get_flux(t, es, 0) * units.second * units.MeV,
            color=to_rgb(color) + (alpha,)
        )
    
    # Only plot the SM flux for the first 8 seconds
    if t > 8 * units.second:
        continue
    
    plt.plot(
        es / units.MeV,
        sm_flux.get_flux(t, es, 0) * units.second * units.MeV,
        color=to_rgb(COLORS[-1]) + (alpha,)   
    )

       
plt.ylabel(
    r"$\frac{\mathrm{d}N}{\mathrm{d}t\mathrm{d}E}~\left[\mathrm{sec}^{-1}\,\mathrm{MeV}^{-1}\right]$",
    fontsize=20
)
plt.xlabel(
    r"$E_{\nu}~\left[\mathrm{GeV}\right]$"
)
plt.ylim(1e1, 1e10)
plt.xlim(es.min() / units.MeV, es.max() / units.MeV)
plt.semilogy()
plt.legend(
    handles=handles,
    ncol=2,
    fontsize=14,
    framealpha=1.0,
    loc=1,
    bbox_to_anchor=(1, 1.075)
)
plt.savefig("../figures/majoran_fluxes.pdf")
plt.show()

# Hits and likelihood plot

## Compute the hits from each flux as a function of time

In [ ]:
dt = 0.01 * units.second

sm_ts, sm_hits = sm_flux.get_hits(
    tmax=100 * units["second"],
    model_file="paper_plots.txt",
    dt=dt
)

bsm_ts = None
bsm_hits = []

for flux in bsm_fluxes:
    bsm_ts_tmp, hits = flux.get_hits(
        model_file="paper_plots.txt",
        tmax=100 * units["second"],
        dt=dt
    )
    
    # Make sure we're not doing something goofy with the times
    if bsm_ts is not None:
        if not np.all(bsm_ts_tmp==bsm_ts):
            raise ValueError()

    bsm_ts = bsm_ts_tmp
    bsm_hits.append(hits)
    

bg_hits = flux.get_background(
    shape=bsm_ts.shape,
    model_file="paper_plots.txt",
    tmax=100 * units["second"],
    dt=dt
)

if np.any(sm_ts!=bsm_ts):
    raise ValueError("Hit times are different !")

## Consider the $\Delta\mathrm{LLH}$ when only considering hits in a certain time window

In [ ]:
times = []
for hits in tqdm(bsm_hits):
    significance, tmins, dts = find_significance_matrix(
        hits,
        sm_hits,
        bg_hits,
        bsm_ts
    )

    significance.T[::-1][-1] = 0
    if np.any(np.isnan(significance)):
        raise ValueError()
    max_significance = significance.max()

    idxs, jdxs = np.where(significance==max_significance)
    
    times.append((tmins[idxs[0]], dts[jdxs[0]]))

## Interpolate $\mathrm{d}N/\mathrm{d}t$ for plotting

In [ ]:
sm_spl = CubicSpline(sm_ts, sm_hits / dt)

logspace_ts = np.logspace(-2, 2, 1000) * units.second

bsm_spls = []
for hits in bsm_hits:
    spl = CubicSpline(bsm_ts, hits / dt)
    bsm_spls.append(spl)

## Make the plots

In [ ]:
fig = plt.figure(figsize=(14, 5))
gs = GridSpec(2, 2, figure=fig, height_ratios=[4,1], hspace=0.06)

ax0 = fig.add_subplot(gs[0, 0])
ax1 = fig.add_subplot(gs[1, 0])
ax2 = fig.add_subplot(gs[:, 1])

# Top left subplot
arrow_kwargs0 = {
    "length_includes_head": True,
    "head_width": 3e4,
    # "head_length": 0.2,
    "facecolor": None,
    "linewidth": 0.0,
    "width": 1e4,
}

ax0.step(logspace_ts / units.second, sm_spl(logspace_ts) * units.second, where="post", color=COLORS[-1], label="Standard Model")
for color, label, spl in zip(COLORS, LABELS, bsm_spls):
    ax0.step(logspace_ts / units.second, spl(logspace_ts) * units.second, where="post", label=label, color=color)
    
# ax0.step(sm_ts / units.second, sm_hits / dt * units.second, where="post", color=COLORS[-1], label="Standard Model")
# for color, label, hits in zip(COLORS, LABELS, bsm_hits):
#     ax0.step(bsm_ts / units.second, hits / dt * units.second, where="post", label=label, color=color)

# ax0.arrow(times[0][0] / units.second, 2.1e5, 0.4, 0, color=COLORS[1], **arrow_kwargs0)
# ax0.arrow((times[0][0] + times[0][1]) / units.second, 2.1e5, -0.4, 0, color=COLORS[0], **arrow_kwargs0)
ax0.arrow(max(times[0][0] / units.second, 1e-2), 2.1e5, 0.006, 0, color=COLORS[0], head_length=0.003, **arrow_kwargs0)
ax0.arrow((times[0][0] + times[0][1]) / units.second, 2.1e5, -0.04, 0, color=COLORS[0], head_length=0.015, **arrow_kwargs0)
ax0.arrow(max(times[1][0] / units.second, 1e-2), 1.8e5, 0.03, 0, color=COLORS[1], head_length=0.013, **arrow_kwargs0)
ax0.arrow((times[1][0] + times[1][1]) / units.second, 1.8e5, -4, 0, color=COLORS[1], head_length=1.3, **arrow_kwargs0)

for (tmin, deltat), color in zip(times, COLORS):
    ax0.axvline(tmin / units.second, zorder=0, color=color)
    ax0.axvline((tmin + deltat) / units.second, zorder=0, color=color)

ax0.loglog()
ax0.set_ylim(1e2, 5e5)
ax0.set_xlim(1e-2, 12)

ax0.set_xticklabels([])
ax0.set_ylabel(r"$\mathrm{d}N_{\mathrm{hit}}/\mathrm{d}t$")

ax0.legend(fontsize=15, bbox_to_anchor=(0.9,1.21), loc=1, framealpha=1.0)

## Bottom left subplot
arrow_kwargs1 = {
    "length_includes_head": True,
    "head_width": 0.03*3,
    # "head_length": 0.2,
    "facecolor": None,
    "linewidth": 0.0,
    "width": 0.01*3,
}

x = sm_spl.integrate(0, 8*units.second)
ax1.step(logspace_ts / units.second, [sm_spl.integrate(0, t) / x for t in logspace_ts], where="post", color=COLORS[2])
for color, label, spl in zip(COLORS, LABELS, bsm_spls):
    x = spl.integrate(0, 100*units.second)
    ax1.step(logspace_ts / units.second, [spl.integrate(0, t) / x for t in logspace_ts], where="post", label=label, color=color)
    
# ax1.step(sm_ts / units.second, np.cumsum(sm_hits) / sm_hits.sum(), where="post", color=COLORS[2])
# for color, label, hits in zip(COLORS, LABELS, bsm_hits):
#     ax1.step(bsm_ts / units.second, np.cumsum(hits) / hits.sum(), where="post", label=label, color=color)


ax1.axvline(times[1][0] / units.second, zorder=0, color=COLORS[1])
ax1.axvline((times[1][0] + times[1][1]) / units.second, zorder=0, color=COLORS[1])
ax1.axvline(times[0][0] / units.second, zorder=0, color=COLORS[0])
ax1.axvline((times[0][0] + times[0][1]) / units.second, zorder=0, color=COLORS[0])

ax1.arrow(max(times[0][0] / units.second, 1e-2), 0.8, 0.006, 0, color=COLORS[0], head_length=0.003, **arrow_kwargs1)
ax1.arrow((times[0][0] + times[0][1]) / units.second, 0.8, -0.04, 0, color=COLORS[0], head_length=0.015, **arrow_kwargs1)

ax1.arrow(max(times[1][0] / units.second, 1e-2), 0.9, 0.03, 0, color=COLORS[1], head_length=0.013, **arrow_kwargs1)
ax1.arrow((times[1][0] + times[1][1]) / units.second, 0.9, -4, 0, color=COLORS[1], head_length=1.3, **arrow_kwargs1)

ax1.set_xlim(1e-2, 12)
ax1.set_ylim(0, 1.2)

ax1.set_xlabel(r"$t - t_{\mathrm{bounce}}$")
ax1.set_ylabel(r"$\mathrm{CDF}$")

ax1.semilogx()

## Right subplot
im = ax2.imshow(
    significance.T[::-1],
    aspect="auto",
    extent=[0, 100, 0.001, 25],
)

ax2.scatter(
    times[1][0] / units.second,
    (times[1][0] + times[1][1]) / units.second,
    marker="*",
    color=COLORS[1],
    s=150,
    label=r"$-2\Delta\mathrm{LLH}_{\mathrm{max}}$"
)

ax2.set_xlabel(r"$t_{\mathrm{start}}~\left[\mathrm{sec}\right]$")
ax2.set_ylabel(r"$\Delta t~\left[\mathrm{sec}\right]$")

ax2.set_xlim(0, 5)
ax2.set_ylim(0, 12)

for x in [3, 6, 9, 12]:
    ax2.plot(np.linspace(0, 1), x - np.linspace(0, 1), color="k", ls="--", lw=1)
    ax2.plot(np.linspace(2.0, 10), x - np.linspace(2.0, 10), color="k", ls="--", lw=1)
    ax2.text(1.1, x-2.1, r"$t_{\mathrm{max}} = %d $" % x, fontsize=12.5, rotation=-18.5)
    
ax2.plot(np.linspace(1.0, 10), 15 - np.linspace(1.0, 10), color="k", ls="--", lw=1)

cbar = plt.colorbar(im, label=r"-2$\Delta \mathrm{LLH}$")
ax2.legend(loc=1, fontsize=15, framealpha=1.0)

plt.savefig("../figures/hits_and_likelihood.pdf")

plt.show()

# Majoran sensitity plot

In [ ]:
dune_sensitivies = np.genfromtxt("./data/dune_sensitivity.csv", delimiter=",")
hk_sensitivies = np.genfromtxt("./data/hyper_k_sensitivity.csv", delimiter=",")

with h5.File("./data/results.h5") as h5f:
    masses = h5f["majoran_sensitivity"]["masses"][:]
    couplings = h5f["majoran_sensitivity"]["couplings"][:]

plt.plot(masses, couplings, label="This work", color="#6c2624")
plt.plot(hk_sensitivies[:, 0], hk_sensitivies[:, 1] / hk_sensitivies[:, 0], label="Hyper-K", color="#eba33e")
plt.plot(dune_sensitivies[:, 0], dune_sensitivies[:, 1] / dune_sensitivies[:, 0], label="DUNE", color="#c2477f")

plt.xlabel(r"$m_{\phi}~\left[\mathrm{MeV}\right]$")
plt.ylabel(r"$g_{\phi}$")

plt.legend(fontsize=15)

plt.xlim(1e-2, 360)

plt.loglog()

plt.savefig("../figures/majoran_sensitivity.pdf")
plt.show()